# Model testing

In [1]:
import sys
import os

cur_dir = os.getcwd()
root_dir = os.path.dirname(cur_dir+"/../../")
sys.path.append(root_dir)

In [3]:
# import the necessary packages
import pandas as pd
from src.data.load_data import load_data
from src.features.labeling import label_encode, one_hot_encode, label_and_one_hot_encode
from src.features.selection import select_features
from src.data.data_scaling import standardize_data, normalize_data
from src.data.split_data import split_data
from src.models.Logistic import train_Logistic
from src.models.evaluate import evaluate_model

In [3]:
# load the dataset
df_wo = load_data()
df_w = load_data(weight=True)

In [4]:
# label encode readmitted column
df_wo = label_encode(df_wo, "readmitted", {'NO': 0, '>30': 1, '<30': 1})

# one hot encode the categorical columns
df_wo = one_hot_encode(df_wo)

In [5]:
# split the data into train and test
X_train_wo, X_test_wo, y_train_wo, y_test_wo = split_data(df_wo, "readmitted")

In [6]:
# standardize the data
X_train_wo, X_test_wo = standardize_data(X_train_wo), standardize_data(X_test_wo)

In [7]:
# train Logistic Regression model
logreg= train_Logistic(X_train_wo, y_train_wo, max_iter=2000)

In [8]:
from src.features.feature_importance import feature_importance_logreg
imp = feature_importance_logreg(logreg, X_train_wo)

In [18]:
from src.features.feature_importance import pick_top_k_features
pick_top_k_features(X_train_wo, imp, 1000)

NameError: name 'X_train_wo' is not defined

In [19]:
from src.models.DecisionTree import train_DecisionTree
from src.features.feature_importance import feature_importance_other, pick_top_k_features

In [6]:
data = label_and_one_hot_encode(load_data())

In [10]:
# split the data into train and test
X_train, X_test, y_train, y_test = split_data(data, "readmitted")

# train Decision Tree model
dtree = train_DecisionTree(X_train, y_train, max_depth=10)

In [11]:
evaluate_model(dtree, X_train, X_test, y_train, y_test)

Train accuracy:  0.6564939075748217
Test accuracy:  0.627644939403865


(0.6564939075748217, 0.627644939403865)

In [12]:
# feature importance
imp = feature_importance_other(dtree, X_train)

In [13]:
imp

,importance
number_inpatient,0.380512
discharge_disposition_id,0.172923
number_diagnoses,0.055411
number_emergency,0.035900
admission_source_id,0.032473
...,...
diag_2_155,0.000000
diag_2_156,0.000000
diag_2_157,0.000000
diag_2_162,0.000000


In [30]:
data = label_and_one_hot_encode(load_data())

X_train, X_test, y_train, y_test = split_data(data, "readmitted")

# pick top k features
X_train = pick_top_k_features(X_train, imp, 1000)
X_test = pick_top_k_features(X_test, imp, 1000)

In [33]:
dtree = train_DecisionTree(X_train, y_train, max_depth=15)

In [34]:
evaluate_model(dtree, X_train, X_test, y_train, y_test)

Train accuracy:  0.7086725812791286
Test accuracy:  0.6199148378643957


(0.7086725812791286, 0.6199148378643957)